In [33]:
import os

import plotly.graph_objects as go
#
# https://plotly.com/python/reference/scatterpolar/
# https://plotly.com/python/reference/layout/polar/
#
import kaleido # needed for svg export

import numpy as np
import pandas as pd

In [34]:
# color settings
#
iph_dark_blue   = '#263473'
iph_darkest_blue = '#1b2450'
iph_light_blue  = '#d5e8f5'
iph_green       = '#9BC552'
iph_yellow      = '#e8cf2f'

leaf_line_light_color   = iph_light_blue
leaf_line_dark_color    = iph_dark_blue
leaf_line_darkest_color = iph_darkest_blue

leaf_fill_light_color   = iph_light_blue
leaf_fill_dark_color    = iph_dark_blue

pistil_color      = iph_green
pistil_line_color = iph_yellow

# Set backgrounds to transparant => rgba(0,0,0,0) 
transparant_bg = 'rgba(0,0,0,0)'

In [35]:
# flower leaves
# =============
#
# in general r = 2 * cos (nmbr_of_leaves * theta) 
#
# r = 2 * cos (3 * theta) 
# generates 3 leaves 
#

datapts = 200  # number points to create the leave

leaves=pd.DataFrame()

number_of_leaves = 3 
max_leaf_radius = 2
pistil_radius = round(max_leaf_radius / 4 , 2) 

theta1 = np.linspace(-np.pi, np.pi, datapts)
theta2 = theta1 + np.pi  # rotate the plot a bit

r = (max_leaf_radius/2) - ( (max_leaf_radius/2) * np.cos( number_of_leaves * theta1)) 

leaves['r'] = r.tolist()
leaves['theta1'] = theta1.tolist()
leaves['theta2'] = theta2.tolist()

In [36]:
# pistils
# =======

pistils=pd.DataFrame()

theta_pistils = [np.pi * 0, 
                 np.pi * 2/3,
                 np.pi * -2/3 ]     # :.

r_pistils     = [pistil_radius,
                 pistil_radius,
                 pistil_radius]

pistils['x'] = theta_pistils
pistils['y'] = r_pistils

In [37]:
def png_fig(df_leaves,df_pistils,width, height, leaf_variant="light",show_grids=True):

    r_max = df_leaves['r'].max() * 1.1  # make radius a little bit larger
    
    # pistil size depends on max of width or height with a minimum of 2
    # pistil size is in points / px
    pistil_size = max([int(max([width, height]) / 10 ), 2])
    if pistil_size <= 3:
        pistil_line_size = 1
    else: 
        pistil_line_size = int(pistil_size / 5)
 
    # leave line width also depends on max of width or height with a minimum of 2
    # leave line width size is in points / px
    leave_line_width = max([int(max([width, height]) / 36 ), 2])
    
    if leaf_variant.lower() == "dark":
        leaf_line_color = leaf_line_light_color
        leaf_fill_color = leaf_fill_dark_color
        
    elif leaf_variant.lower() == "darkest":
        leaf_line_color = leaf_line_darkest_color
        leaf_fill_color = leaf_fill_dark_color
        
    else:
        leaf_line_color = leaf_line_dark_color
        leaf_fill_color = leaf_fill_light_color
    
    fig = go.Figure()

    fig.update_layout(
        showlegend = False,

        # https://plotly.com/python/setting-graph-size/
        autosize=False,
        width=width,
        height=height,
        margin=dict(
            l=2,
            r=2,
            b=2,
            t=2,
            pad=0
        ),
        
        paper_bgcolor = transparant_bg,
        plot_bgcolor = transparant_bg,
        
        polar = dict(
          
          # https://plotly.com/python/reference/layout/polar/#layout-polar-radialaxis
          radialaxis = dict(
              tickfont = dict(
                  size=8,
              ), 
              range=[0,r_max],
              linecolor = 'darkgrey',
              linewidth = 1,
             
              gridcolor = 'darkgrey',
              gridwidth = 1,
              
              showticklabels = False,
              showgrid = show_grids,
              showline = show_grids,
              
              layer="below traces",
          ),
            
          # https://plotly.com/python/reference/layout/polar/#layout-polar-angularaxis 
          angularaxis = dict(
              tickfont = dict(
                  size=10,
              ), 
              #rotation=90, # start position of angular axis
              #direction="clockwise",
              linecolor = 'darkgrey',
              linewidth = 1,
              
              gridcolor='darkgrey',
              gridwidth=1,
              
              showticklabels = False,
              showgrid = show_grids,
              showline = show_grids,
              
              layer="below traces",
          ),

          bgcolor = transparant_bg,
        )
    )

    fig.add_trace(
        # https://plotly.com/python/reference/scatterpolar/
        go.Scatterpolar(
            name = 'THETA1',

            # default is degrees https://plotly.com/python/reference/scatterpolar/#scatterpolar-thetaunit
            thetaunit = 'radians',

            r = df_leaves['r'],
            theta = df_leaves['theta1'],

            mode = 'lines',
            line = dict(
                color = leaf_line_color,
                width = leave_line_width,
            ),

            fill = "toself",
            fillcolor = leaf_fill_color,
            opacity = 0.8,
        )
    )

    fig.add_trace(
        # https://plotly.com/python/reference/scatterpolar/
        go.Scatterpolar(
            name = 'THETA2',
            
            # default is degrees https://plotly.com/python/reference/scatterpolar/#scatterpolar-thetaunit
            thetaunit = 'radians', 

            r = df_leaves['r'],
            theta = df_leaves['theta2'],

            mode = 'lines',
            line = dict(
                color = leaf_line_color,
                width = leave_line_width,
            ),

            fill = "toself",
            fillcolor = leaf_fill_color,
            opacity = 0.8,
       )
    )

    # Add pistils
    # ===========
    fig.add_trace(go.Scatterpolar(
            r = df_pistils['y'],
            theta = df_pistils['x'],
            mode = 'markers',
            name = 'PISTIL',
            marker= dict(
                size = pistil_size,    
                color = pistil_color,
                opacity=1,
                line=dict(
                    color=pistil_line_color,
                    width=pistil_line_size),
    #              gradient=dict(
    #                  color=pistil_color,
    #                  type='none',
    #             ),
            ),
            # default is degrees https://plotly.com/python/reference/scatterpolar/#scatterpolar-thetaunit
            thetaunit = 'radians'
       )
    )

    return fig

In [38]:
outputs= [
    {"img_w":64, "img_h":64, "variant":"light", "grids":False},
    {"img_w":64, "img_h":64, "variant":"dark", "grids":False},
    {"img_w":80, "img_h":80, "variant":"light", "grids":False},
    {"img_w":80, "img_h":80, "variant":"dark", "grids":False},
    {"img_w":1024, "img_h":1024, "variant":"light", "grids":True},
    {"img_w":1024, "img_h":1024, "variant":"dark", "grids":True},
    {"img_w":1024, "img_h":1024, "variant":"light", "grids":False},
    {"img_w":1024, "img_h":1024, "variant":"dark", "grids":False},
    {"img_w":1024, "img_h":1024, "variant":"darkest", "grids":True},
    {"img_w":1024, "img_h":1024, "variant":"darkest", "grids":False},
]

for output in outputs:
    if output["grids"]:
        grid_str="with_grid"
    else:
        grid_str="no_grid"

    a_fig = png_fig(df_leaves=leaves,
                df_pistils=pistils,
                width=output["img_w"],
                height=output["img_h"],
                leaf_variant=output["variant"],
                show_grids=output["grids"])


    if not os.path.exists('./assets/'):
        os.makedirs('./assets/')
    # https://plotly.com/python/static-image-export/

    a_fig.write_image('./assets/ipheion-logo-2022-'+
                      output["variant"]+'-'+
                      grid_str+'-'+
                      str(output["img_w"])+'x'+
                      str(output["img_h"])+'.png')
    
    a_fig.write_image('./assets/ipheion-logo-2022-'+
                      output["variant"]+'-'+
                      grid_str+'-'+
                      str(output["img_w"])+'x'+
                      str(output["img_h"])+'.svg')

    a_fig.show()


In [39]:
# https://www.janvas.com/janvas-app

In [40]:
# https://favicon.io/favicon-converter/